# Description

It projects input data into a UMAP representation.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from IPython.display import display

import pandas as pd
import umap
import matplotlib.pyplot as plt
import seaborn as sns

import conf
from utils import generate_result_set_name

# Settings

In [3]:
INPUT_FILEPATH_STEM = 'diseases_only-smultixcan-efo_partial-mashr-zscores'

In [4]:
INPUT_FILEPATH = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'z_score_std',
    f'z_score_std-{INPUT_FILEPATH_STEM}.pkl',
).resolve()
display(INPUT_FILEPATH)

input_filepath_stem = INPUT_FILEPATH.stem
display(input_filepath_stem)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/z_score_std/z_score_std-diseases_only-smultixcan-efo_partial-mashr-zscores.pkl')

'z_score_std-diseases_only-smultixcan-efo_partial-mashr-zscores'

In [5]:
# number of components to use in the dimensionality reduction step
DR_OPTIONS = {
    'n_components': [5, 10, 20, 30, 40, 50],
    'metric': 'euclidean',
    'n_neighbors': 15,
    'random_state': 0,
}

In [6]:
# output dir for this notebook
RESULTS_DIR = Path(
    conf.RESULTS["DATA_TRANSFORMATIONS_DIR"],
    'umap'
).resolve()
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

display(RESULTS_DIR)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap')

In [7]:
# dictionary containing all options/settings (used to generate filenames)
ALL_OPTIONS = DR_OPTIONS.copy()
# ALL_OPTIONS['proj_percentile'] = PERCENTILE_NAME

display(ALL_OPTIONS)

{'n_components': [5, 10, 20, 30, 40, 50],
 'metric': 'euclidean',
 'n_neighbors': 15,
 'random_state': 0}

# Load input file

In [8]:
data = pd.read_pickle(INPUT_FILEPATH)

In [9]:
data.shape

(538, 6452)

In [10]:
data.head()

gene_name,ENSG00000183087,ENSG00000157227,ENSG00000096696,ENSG00000175130,ENSG00000113140,ENSG00000117984,ENSG00000116016,ENSG00000129116,ENSG00000134686,ENSG00000108679,...,ENSG00000111716,ENSG00000166796,ENSG00000114331,ENSG00000131584,ENSG00000165410,ENSG00000172757,ENSG00000147862,ENSG00000008323,ENSG00000167083,ENSG00000149257
vascular disease AND intestinal disease,0.847454,1.009161,-0.834057,-0.070589,-0.329654,1.339072,-1.317406,-0.153223,0.208080,1.305476,...,-0.862693,0.178475,-0.867851,-0.318883,0.023148,0.121395,0.106954,0.282147,-0.811827,0.413489
"osteoarthritis, knee",-0.949368,2.236648,0.059025,0.312724,-1.303170,0.827997,2.295848,-0.118700,1.478273,-0.048731,...,-0.900876,1.239900,-0.357637,-0.510200,-0.233842,-0.130737,-0.066291,-0.470850,0.133876,0.992046
carpal tunnel syndrome,3.451503,0.179316,1.656231,-0.075898,0.546487,0.182674,0.303961,-0.571854,-0.822994,0.212698,...,1.314737,-0.700739,-0.570497,0.672079,-0.255689,0.473933,-0.319597,2.733735,1.247620,-0.433203
gastritis,-0.301861,0.026726,-1.095334,0.238529,-1.060668,-1.277446,1.096986,-0.490961,0.388828,2.672629,...,1.686590,-0.032626,-0.554427,0.406311,-0.725370,-0.700858,-0.669952,2.088057,-0.195165,1.154248
neoplasm,3.186200,-0.215859,-0.640350,0.748803,-1.183247,-0.773517,0.111114,-0.579060,-0.909444,-0.860679,...,0.849432,0.533190,0.851463,0.588349,-1.023563,0.226923,1.508859,0.368243,0.636630,0.131323


# UMAP

In [11]:
def get_umap_proj(orig_data, options):
    umap_options = {k:v for k, v in options.items() if k in DR_OPTIONS}
    display(f'UMAP options: {umap_options}')
    umap_obj = umap.UMAP(**umap_options)
    umap_obj = umap_obj.fit(orig_data)
    umap_data = umap_obj.transform(orig_data)
    return pd.DataFrame(
        data=umap_data,
        index=orig_data.index.copy(),
        columns=[f'UMAP{i+1}' for i in range(umap_data.shape[1])]
    )

In [12]:
# for n_comp, n_neigh in product(DR_OPTIONS['n_components'], DR_OPTIONS['n_neighbors']):
for n_comp in DR_OPTIONS['n_components']:
    print(f'# components: {n_comp}')
    
    options = ALL_OPTIONS.copy()
    options['n_components'] = n_comp
    
    dr_data = get_umap_proj(data, options)
    
    display(dr_data.shape)
    assert dr_data.shape == (data.shape[0], n_comp)
    
    display(dr_data.iloc[:, 0:5].describe())
    
    # save
    output_file = Path(
        RESULTS_DIR,
        generate_result_set_name(
            options,
            prefix=f'umap-{input_filepath_stem}-',
            suffix='.pkl'
        )
    ).resolve()
    display(output_file)
    
    dr_data.to_pickle(output_file)
    
    print('\n')

# components: 5


"UMAP options: {'n_components': 5, 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}"

(538, 5)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,538.000000,538.000000,538.000000,538.000000,538.000000
mean,9.916106,6.716135,4.584635,8.255410,5.665169
std,2.427202,1.323215,0.595043,0.551621,1.298313
min,4.604455,4.508282,3.274413,7.229296,3.943371
25%,7.029998,5.744145,4.100205,7.738104,4.694282
50%,11.228889,6.157133,4.610241,8.288859,5.102781
75%,11.679579,8.278051,5.017382,8.714386,7.181852
max,12.785789,9.396283,6.110092,9.348644,8.177178


PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-z_score_std-diseases_only-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_5-n_neighbors_15-random_state_0.pkl')

...



# components: 50


"UMAP options: {'n_components': 50, 'metric': 'euclidean', 'n_neighbors': 15, 'random_state': 0}"

(538, 50)

,UMAP1,UMAP2,UMAP3,UMAP4,UMAP5
count,538.000000,538.000000,538.000000,538.000000,538.000000
mean,7.985046,2.601374,4.829426,8.431797,6.059007
std,2.297926,0.278715,0.191931,0.181283,0.560595
min,2.842567,1.874109,4.428329,8.073075,4.749494
25%,5.216704,2.347823,4.660509,8.297781,5.464290
50%,9.395594,2.698653,4.820894,8.411202,6.322286
75%,9.576528,2.803142,4.967501,8.572313,6.481516
max,10.044398,3.033765,5.325864,8.810924,6.745290


PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/data_transformations/umap/umap-z_score_std-diseases_only-smultixcan-efo_partial-mashr-zscores-metric_euclidean-n_components_50-n_neighbors_15-random_state_0.pkl')